## 추진배경

* 업무상 공용ID가 3개로 나뉘어있으나, 모두 공동인증서 로그인으로 접속에 불편함이 있음 (ID, PW, 공동인증서PW)
  * 발급현황확인, 발급거절시 사유확인 후 보완 등 진행
* 발급거절(오류통보) 사유확인시, 건별로 클릭하여 메뉴진입 필요
* 월마다 20여개 페이지의 표를 복사해서 가공하는 작업 수행중(데이터 Merge, 필터링 등)
* 각 포인트에 대해 대응할 수 있는 여러 기능을 탑재한 통합관리 Tool 제작
  * 주기적으로 3개 ID로 접속하여, 주요정보를, ID에 대응되는 테이블에 db형태로 저장(공용PC)
  * 유저는 streamlit으로 제작한 사이트에 접속해서 필요한 내용 확인/검색
  * 희망하는 경우, 대표Invoice번호를 등록해두면 발급완료/발급거절(보완) 상황발생시 toast알림
  * 월마다 가공하는 데이터에 대해서는, 별도버튼으로 db추출 후 가공완료한 데이터 제공
  * ID에 대응하는 로그인 버튼을 누르면 자동로그인 후 작업창을 띄워주는 기능 제공

>
**[용어설명]**<br>
**COO** : 원산지증명서, Country of Origin<br>

## 효과
* 불편했던 로그인 작업 수요감소(필요 데이터의 streamlit 대시보드 제공, toast알림 제공), 로그인 편의성 증가
* 월 가공작업 삭제(sql query로 추출시 원하는 형태로 저장, streamlit에서 query문 조건변경 가능[날짜조건, 파일경로 등])

## github repository
[관련 github레포](https://github.com/KR9268/-P-MonitoringCOO)

## [세부내용] 구현내용 & 사용한 언어/패키지 등
(1) 저장할 db는 `sqlite3`으로 테이블 생성, 컬럼지정 등을 수행함 (컬럼별 조건은 하단 참조)
  * 접수번호 varchar PRIMARY KEY ,<br> → **대표Invoice와 고민했는데, 100% 유일값이라 Primary로 지정** 
     증명서종류 varchar, <br>
     대표Invoice varchar(10),<br> 
     접수일시 datetime, <br>
     처리상태 varchar, <br>
     Remark varchar
(2) 각 기능은 아래의 파일로 나누어 개인/공용PC에서 실행
  * **MonitoringCOO(기본파일)** : `streamlit`활용한 UI, json/pickle파일 읽기, 유저의 자동로그인, 월추출 데이터 저장 등
    * 마지막 스크레핑 시점을 표기하여 얼마나 최신화된 데이터인지 유저에게 공유
  * **MonitoringCOO_crawler** : `selenium`으로 스크레핑, 스크레핑작업에 필요한 로그인 기능(`pyautogui`, `pywin32`로 이미지/키/윈도우 인식)
    * 유저가 **기본파일**에서 로그인기능을 사용하는 경우, 이 파일에서 import해서 사용하고 **코드는 여기서 통합관리**
    * 스크레핑작업은 기본적으로 **Scheduler**파일에서 실행되지만, 필요시 이 파일을 실행하여 수동 스크레핑 (**코드는 여기서 통합관리**)
  * **MonitoringCOO_push** : **기본파일**에서 유저가 등록해둔 대표Invoice번호를 db에서 조회하여, `win11toast`로 알림
                         (처음에는 파이썬과 호환성/속도가 좋은 pickle/list로 관리하고자 했으나, 사용자ID등 추가정보 관리가 필요하여 json/dict로 관리)
  * **Scheduler** : 스크레핑 주기/시간을 관리하는 파일. 주로 공용PC에서 작업
                    (9~17시 이후엔 데이터변경이 없으므로 이 시간대에만 작동하도록 설정, 서버설정 등을 고려하여 작업주기 반영 예정)